In [1]:
'''!pip install -q -U sqlalchemy 
!pip install -q -U pyodbc
!pip install -q -U requests
!pip install -q -U pandas
!pip install -q -U xmltodict
'''
#!pip install sqlite3

import json
import sqlite3
import pyodbc
import sqlalchemy 
import xml.etree.ElementTree as ET
import requests
import pprint
import pandas as pd
from sqlalchemy import create_engine, text

#import duckdb
#import pandas as pd

In [2]:
# Caminho para o banco de dados SQLite
db_path = r'C:\PROJETOS\BASE_CNPJ\db_cadastro_receita'

# Conexão com o banco de dados SQLite
def conectar_banco(db_path):
    conn = sqlite3.connect(db_path)
    return conn

conn = conectar_banco(db_path)
cursor = conn.cursor()

Lendo do dicionario EMPRESA DATASET gov.br

In [3]:
def ler_arquivos_concatenados(arquivos, caminho_base, chunksize=5000):

    # Defina os tipos de dados esperados para as colunas problemáticas
    dtype_spec = {
        0: str,    
        1: str,
        2: str,
        3: str,
        4: str,
        5: float,
        6: str
    }

    for arquivo in arquivos:
        file_path = f"{caminho_base}\\{arquivo}"

        print(f"Processando o arquivo: {file_path}")
        
        # Iterar sobre o arquivo em chunks
        for chunk in pd.read_csv(
            file_path, 
            delimiter=';', 
            quotechar='"', 
            encoding='latin1',
            header=None,
            dtype=dtype_spec,
            low_memory=False,
            chunksize=chunksize
        ):
            chunk = chunk.fillna("")
            # Renomear colunas
            chunk = chunk.rename(columns={
                0: 'CNPJ_BASICO',
                1: 'RAZAO_SOCIAL',
                2: 'NATUREZA_JURIDICA',
                3: 'QUALIFICACAO_RESPONSAVEL',
                4: 'CAPITAL_SOCIAL',
                5: 'PORTE',
                6: 'ENTE_FEDERATIVO'
            })
            # Inserir chunk no banco de dados
            #chunk.to_sql('EMPRESAS', con=engine, index=False, if_exists='append')
            chunk.to_sql('EMPRESAS', con=conn, index=False, if_exists='append')
            #print(f"Chunk de {chunksize} linhas inserido com sucesso.")
    conn.close()



In [4]:
# Lista de arquivos
arquivos = [
    "K3241.K03200Y1.D40608.EMPRECSV",
    "K3241.K03200Y2.D40608.EMPRECSV",
    "K3241.K03200Y3.D40608.EMPRECSV",
    "K3241.K03200Y4.D40608.EMPRECSV",
    "K3241.K03200Y5.D40608.EMPRECSV",
    "K3241.K03200Y6.D40608.EMPRECSV",
    "K3241.K03200Y7.D40608.EMPRECSV",
    "K3241.K03200Y8.D40608.EMPRECSV",
    "K3241.K03200Y9.D40608.EMPRECSV"
]

# Caminho base onde os arquivos estão localizados
caminho_base = r'C:\PROJETOS\BASE_CNPJ\EXTRAIDO\EMPRESA'

# Chamada da função para processar e inserir os dados em chunks
#ler_arquivos_concatenados(arquivos, caminho_base)

Lendo do dicionario ESTABELECIMENTOS DATASET gov.br

In [5]:
def processar_csv(arquivos, caminho_base, chunksize=5000):

    conn = conectar_banco(db_path)
    cursor = conn.cursor()
    
    # Defina os tipos de dados esperados para as colunas problemáticas
    dtype_spec = {
        0: str,    
        1: str,
        2: str,
        3: str,
        #4: str,
        5: str,
        6: str,     
        10: str,
        11: str,  
        12: str,     
        18: str,    
        19: str,
        20: str,
        27: str,
    }

    # Lista de colunas que queremos ler
    colunas = [
        0,
        1,
        2,
        3,
        #4,
        5,
        6,
        10,
        11,
        12,
        18,
        19,
        20,
        27
    ]
    
    for arquivo in arquivos:
        file_path = f"{caminho_base}\\{arquivo}"
        print(f"Processando o arquivo: {file_path}")
    # Leia o arquivo com as colunas especificadas

        # Iterar sobre o arquivo em chunks
        for chunk in pd.read_csv(
            file_path, 
            delimiter=';', 
            quotechar='"', 
            encoding='latin1',
            header=None,
            dtype=dtype_spec,
            usecols=colunas,
            low_memory=False,
            chunksize=chunksize
        ):
            #removendo nulos
            chunk = chunk.fillna("")
            
            # Renomear colunas
            chunk = chunk.rename(columns={
                0: 'CNPJ BÁSICO',
                1: 'CNPJ ORDEM',
                2: 'CNPJ DV',
                3: 'IDENTIFICADOR MATRIZ/FILIAL',
                #4: 'NOME FANTASIA',
                5: 'SITUAÇÃO CADASTRAL',
                6: 'DATA SITUAÇÃO CADASTRAL',
                10: 'DATA DE INÍCIO ATIVIDADE',
                11: 'CNAE FISCAL PRINCIPAL',
                12: 'CNAE FISCAL SECUNDÁRIA',
                18: 'CEP',
                19: 'UF',
                20: 'MUNICÍPIO',
                27: 'CORREIO ELETRÔNICO',
            })

            chunk.to_sql('ESTABELECIMENTOS', con=conn, index=False, if_exists='append')
            #print(f"Chunk de {chunksize} linhas inserido com sucesso.")
    conn.close()


In [6]:
# Lista de arquivos
arquivos = [
    "K3241.K03200Y0.D40608.ESTABELE",
    "K3241.K03200Y1.D40608.ESTABELE",
    "K3241.K03200Y2.D40608.ESTABELE",
    "K3241.K03200Y3.D40608.ESTABELE",
    "K3241.K03200Y4.D40608.ESTABELE",
    "K3241.K03200Y5.D40608.ESTABELE",
    "K3241.K03200Y6.D40608.ESTABELE",
    "K3241.K03200Y7.D40608.ESTABELE",
    "K3241.K03200Y8.D40608.ESTABELE",
    "K3241.K03200Y9.D40608.ESTABELE"
]

# Caminho base onde os arquivos estão localizados
caminho_base = r'C:\PROJETOS\BASE_CNPJ\EXTRAIDO\ESTABELECIMENTOS'


# Chamada da função para processar e inserir os dados em chunks
#processar_csv(arquivos, caminho_base)

MUNICIPIO

In [ ]:
# Defina o caminho do arquivo
file_path = r'C:\PROJETOS\BASE_CNPJ\EXTRAIDO\COMPLEMENTARES\F.K03200$Z.D40608.MUNICCSV'

# Defina os tipos de dados esperados para as colunas problemáticas
dtype_spec = {
    0: str,    
    1: str
}




df_municipios = pd.read_csv(
    file_path, 
    delimiter=';', 
    quotechar='"', 
    encoding='latin1',
    header=None,
    dtype=dtype_spec,
    #usecols=colunas,
    low_memory=False
)

# Renomear colunas
df_municipios = df_municipios.rename(columns={
    0: 'CODIGO',
    1: 'MUNICIPIO'
})

conn = conectar_banco(db_path)
cursor = conn.cursor()

#df_municipios.to_sql('MUNICIPIO', con=conn, index=False, if_exists='append')

conn.close()

#display(df_municipios)

CNAE

In [15]:
# Defina o caminho do arquivo
file_path = r'C:\PROJETOS\BASE_CNPJ\EXTRAIDO\COMPLEMENTARES\F.K03200$Z.D40608.CNAECSV'

# Defina os tipos de dados esperados para as colunas problemáticas
dtype_spec = {
    0: str,    
    1: str
}

df_cnae = pd.read_csv(
    file_path, 
    delimiter=';', 
    quotechar='"', 
    encoding='latin1',
    header=None,
    dtype=dtype_spec,
    #usecols=colunas,
    low_memory=False
)

# Renomear colunas
df_cnae = df_cnae.rename(columns={
    0: 'CODIGO',
    1: 'CNAE'
})

conn = conectar_banco(db_path)
cursor = conn.cursor()

df_cnae.to_sql('CNAE', con=conn, index=False, if_exists='append')

conn.close()

#display(df_municipios)

PAISES

In [16]:
# Defina o caminho do arquivo
file_path = r'C:\PROJETOS\BASE_CNPJ\EXTRAIDO\COMPLEMENTARES\F.K03200$Z.D40608.PAISCSV'

# Defina os tipos de dados esperados para as colunas problemáticas
dtype_spec = {
    0: str,    
    1: str
}

df_pais = pd.read_csv(
    file_path, 
    delimiter=';', 
    quotechar='"', 
    encoding='latin1',
    header=None,
    dtype=dtype_spec,
    #usecols=colunas,
    low_memory=False
)

# Renomear colunas
df_pais = df_pais.rename(columns={
    0: 'CODIGO',
    1: 'PAIS'
})

conn = conectar_banco(db_path)
cursor = conn.cursor()

df_pais.to_sql('PAIS', con=conn, index=False, if_exists='append')

conn.close()

#display(df_municipios)

SITUAÇÃO CADASTRO

In [18]:
# Defina o caminho do arquivo
file_path = r'C:\PROJETOS\BASE_CNPJ\EXTRAIDO\COMPLEMENTARES\F.K03200$Z.D40608.MUNICCSV'

# Defina os tipos de dados esperados para as colunas problemáticas
dtype_spec = {
    0: str,    
    1: str
}

df_situação_cadastro = pd.read_csv(
    file_path, 
    delimiter=';', 
    quotechar='"', 
    encoding='latin1',
    header=None,
    dtype=dtype_spec,
    #usecols=colunas,
    low_memory=False
)

# Renomear colunas
df_situação_cadastro = df_situação_cadastro.rename(columns={
    0: 'CODIGO',
    1: 'DESCRICAO'
})

conn = conectar_banco(db_path)
cursor = conn.cursor()

df_situação_cadastro.to_sql('SITUACAO_CADASTRO', con=conn, index=False, if_exists='append')

conn.close()

#display(df_municipios)